# Préparation de l'espace de travail

## Importations

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Lecture des bases de données

In [ ]:
# Lecture des différentes feuilles du Excel medecins
df_medecinsEffectif = pd.read_excel("medecins.xlsx", sheet_name=1)
df_medecinsAge = pd.read_excel("medecins.xlsx", sheet_name=2)
df_medecinsDensite = pd.read_excel("medecins.xlsx", sheet_name=3)
df_medecinsEffectifsParSpe = pd.read_excel("medecins.xlsx", sheet_name=4)
df_medecinsEffectifsParLieu = pd.read_excel("medecins.xlsx", sheet_name=5)
df_medecinsNombreActivites = pd.read_excel("medecins.xlsx", sheet_name=6)

# Lecture du csv santePublique
df_santePublique = pd.read_csv("santePublique.csv", sep=";")

# Visualisation des bases de données

## Base santePublique

In [ ]:
print(df_santePublique.columns)

# Tri à plat de la variable patho_niv1
freq = df_santePublique["patho_niv1"].value_counts()
print(freq)

# plt.figure(figsize=(12, 5))
# sns.barplot(x=freq.index, y=freq.values)
# plt.xlabel("Valeurs")
# plt.xticks(rotation=90)
# plt.ylabel("Fréquence")
# plt.title("Fréquence de chaque valeur")
# plt.show()

In [ ]:
df_santePublique['dept'].value_counts()
df_santePublique.head()

df_santePublique.loc[(df_santePublique['dept']=='11') & (df_santePublique['annee']==2022) & (df_santePublique['sexe']==9)]

In [ ]:
# Liste des variables à garder constantes pour la création de nos variables agrégées (tout âge confondu)
colonnes = ['annee', 'patho_niv1', 'patho_niv2', 'patho_niv3', 'sexe', 'dept']

# Création des variables Ntop_agrege et Npop_agrege qui donnent les effectifs agrégés pour chaque pathologie, tout âge confondu.
df_santePublique['Ntop_agrege'] = df_santePublique.groupby(colonnes, dropna = False)['Ntop'].transform(sum)
df_santePublique['Npop_agrege'] = df_santePublique.groupby(colonnes, dropna = False)['Npop'].transform(sum)

# Création de la variable prev_agrege qui donne la prévalence pour chaque pathologie, tout âge confondu
df_santePublique['prev_agrege'] = df_santePublique['Ntop_agrege'] / df_santePublique['Npop_agrege']